In [1]:
import os
import glob
import pyart
import numpy as np
from pyhail import hsda, hdr, mesh
from cpol_processing import processing as cpol_prc
from datetime import datetime
from multiprocessing import Pool


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/cpol_processing-1.0-py3.6.egg/cpol_processing/production.py:28: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in la

In [2]:
#paths
vol_path         = '/g/data1a/rq0/odim_archive/hsda-testing/20141127_mdv'
out_path         = '/g/data1a/rq0/odim_archive/hsda-testing/20141127_processed'
sonde_ffn        = '/g/data1a/rq0/odim_archive/hsda-testing/snding_data/YBBN_20141127_00.nc'
radar_name       = 'CP2'

#field names (used to map to radar object fields)
fieldnames  = {'dbzh':'DBZ_CORR', 'zdr':'ZDR_CORR', 'rhv':'RHOHV_CORR', 'hca':'CLASS','hsda': 'CLASS_HSDA'}

#hsda vars
hca_hail_idx = [9] #index of hail classes in HCA
hca_hsda_idx = [11,12,13] #index to use for HSDA small, large and giant classes on top of HCA data
dzdr         = 0

#multiprocessing
ncpu         = 16

#TODO
#Implement dzdr
#Implement beam blockage mapping

In [3]:
def chunks(l, n):
    """
    Yield successive n-sized chunks from l.
    From http://stackoverflow.com/a/312464
    """
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [4]:
def worker(radar_file_name, radar_name):
        ###########################################################
    # Load file
    ###########################################################
    #load radar object
    if ".h5" in radar_file_name:
        radar = pyart.aux_io.read_odim_h5(radar_file_name)
    else:
        radar = pyart.io.read(radar_file_name)
    #extract date    
    date_str = radar.time['units'][-20:]
    dt       = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')    
    
    #fix field names
    radar.add_field('RHOHV', radar.fields.pop('cross_correlation_ratio'))
    radar.add_field('DBZH', radar.fields.pop('reflectivity'))
    radar.add_field('KDP', radar.fields.pop('specific_differential_phase'))
    radar.add_field('ZDR', radar.fields.pop('differential_reflectivity'))
    radar.add_field('PHIDP', radar.fields.pop('differential_phase'))
    radar.add_field('NCP', radar.fields.pop('normalized_coherent_power')) 
    
    ###########################################################
    # Filtering
    ###########################################################
    
    #rhohv gatefilter
    gatefilter = pyart.filters.GateFilter(radar)
    gatefilter.exclude_below('RHOHV', 0.7)
    
    #rhohv texture filtering
    #gatefilter = pyart.filters.moment_and_texture_based_gate_filter(
    
    ###########################################################
    # Correction
    ###########################################################
    
    #build temp information
    height, temperature, snr = cpol_prc.radar_codes.snr_and_sounding(radar, sonde_ffn, refl_field_name='DBZH', temp_field_name='temp')
    radar.add_field('TEMPERATURE', temperature, replace_existing=True)
    radar.add_field('HEIGHT', height, replace_existing=True)
    radar.add_field('SNR', snr, replace_existing=True)
    
    #RHOHV Noise correct
    rho_corr = cpol_prc.radar_codes.correct_rhohv(radar, rhohv_name='RHOHV', snr_name='SNR')
    radar.add_field_like('RHOHV', 'RHOHV_CORR', rho_corr, replace_existing=True)
    
    #ZDR Noise Correct
    corr_zdr = cpol_prc.radar_codes.correct_zdr(radar, zdr_name='ZDR', snr_name='SNR')
    radar.add_field_like('ZDR', 'ZDR_CORR', corr_zdr, replace_existing=True)
    
    #unfold phidp
    phi_unfold = cpol_prc.phase.unfold_raw_phidp(radar, refl_field='DBZH', ncp_field='NCP', rhv_field='RHOHV_CORR', phi_name="PHIDP")
    radar.add_field("PHI_UNF", phi_unfold, replace_existing=True)

    #recalculate phidp
    phimeta, kdpmeta = cpol_prc.phase.phidp_bringi(radar, gatefilter, refl_field='DBZH', ncp_name='NCP', rhohv_name='RHOHV_CORR', unfold_phidp_name="PHI_UNF")
    radar.add_field('PHIDP_BRINGI', phimeta, replace_existing=True)
    radar.add_field('KDP_BRINGI', kdpmeta, replace_existing=True)
    radar.fields['PHIDP_BRINGI']['long_name'] = "corrected_differential_phase"
    radar.fields['KDP_BRINGI']['long_name'] = "corrected_specific_differential_phase"

    ###########################################################
    # Attenuation
    ###########################################################
    
    #ZH attenuation correction
    atten_spec, zh_corr = cpol_prc.attenuation.correct_attenuation_zh_pyart(radar, refl_field='DBZH', ncp_field='NCP', rhv_field='RHOHV_CORR', phidp_field='KDP_BRINGI')
    radar.add_field('DBZ_CORR', zh_corr, replace_existing=True)
    radar.add_field('specific_attenuation_reflectivity', atten_spec, replace_existing=True)    
    
    #ZDR attenuation correction
    atten_spec_zdr, zdr_corr = cpol_prc.attenuation.correct_attenuation_zdr(radar, zdr_name='ZDR_CORR', kdp_name='KDP_BRINGI', alpha=0.016)
    radar.add_field_like('ZDR', 'ZDR_CORR', zdr_corr, replace_existing=True)
    radar.add_field('specific_attenuation_differential_reflectivity', atten_spec_zdr,
                    replace_existing=True)
    
    ###########################################################
    # Apply filter
    ###########################################################
    
    #apply rhohv filter
    radar.fields['DBZ_CORR']['data']   = cpol_prc.filtering.filter_hardcoding(radar.fields['DBZ_CORR']['data'], gatefilter)
    radar.fields['ZDR_CORR']['data']   = cpol_prc.filtering.filter_hardcoding(radar.fields['ZDR_CORR']['data'], gatefilter)
    radar.fields['KDP_BRINGI']['data'] = cpol_prc.filtering.filter_hardcoding(radar.fields['KDP_BRINGI']['data'], gatefilter)
    radar.fields['RHOHV']['data']      = cpol_prc.filtering.filter_hardcoding(radar.fields['RHOHV']['data'], gatefilter)
    
    ###########################################################
    # Classifications
    ###########################################################
    
    #CSU HCA
    hydro_class = cpol_prc.hydrometeors.hydrometeor_classification(radar, refl_name='DBZ_CORR', zdr_name='ZDR_CORR', 
                                                                   kdp_name='KDP_BRINGI', rhohv_name='RHOHV_CORR', 
                                                                   height_name='HEIGHT', temperature_name='TEMPERATURE')
    radar.add_field('CLASS', hydro_class, replace_existing=True)    
    
    #HSDA
    hsda_data = hsda.main(radar,sonde_ffn,fieldnames,hca_hail_idx,hca_hsda_idx,dzdr)
    the_comments = "1: Drizzle; 2: Rain; 3: Ice Crystals; 4: Aggregates; " +\
                   "5: Wet Snow; 6: Vertical Ice; 7: LD Graupel; 8: HD Graupel; 9: NOT USED; 10: Big Drops" +\
                   "11: Small Hail (< 25 mm); 12: Large Hail (25 - 50 mm); 13: Giant Hail (> 50 mm)"
    hsda_meta   = {'data': hsda_data, 'units': ' ', 'long_name': 'Hydrometeor classification + HSDA',
                  'standard_name': 'Hydrometeor_ID_HSDA', 'comments': the_comments}
    radar.add_field('HSDA', hsda_meta, replace_existing=True) 
    
    #HDR
    hdr_data = hdr.main(radar,fieldnames)
    the_comments = "Applies the transform from hdr to mm used by Depue et al. 2009"
    hdr_meta     = {'data': hdr_data, 'units': 'mm', 'long_name': 'Hail Differential Reflectivity',
                  'standard_name': 'HDR', 'comments': the_comments}
    radar.add_field('HDR', hdr_meta, replace_existing=True)
    
    ###########################################################
    # CFradial output
    ###########################################################
    
    #write radar object to file
    out_fn  = '_'.join([radar_name, dt.strftime('%Y%m%d_%H%M%S'), 'processed']) + '.nc'
    out_ffn = '/'.join([out_path, out_fn])
    try:
        os.remove(out_ffn)
    except OSError:
        pass  
    pyart.io.write_cfradial(out_ffn, radar)
    
    print('completed volume ' + out_ffn)
    
    ###########################################################
    # Gridded Processing and Output
    ###########################################################
    
    out_fn  = '_'.join([radar_name, dt.strftime('%Y%m%d_%H%M%S'), 'meshgrids']) + '.nc'
    out_ffn = '/'.join([out_path,out_fn])
    try:
        os.remove(out_ffn)
    except OSError:
        pass
    #MESH
    mesh.main(radar, fieldnames, out_ffn, sonde_ffn)
    
    print('completed grid ' + out_ffn)

In [ ]:
#index vol files
vol_filelist = sorted(glob.glob(vol_path + '/*.mdv'))

# Cutting the file list into smaller chunks. (The multiprocessing.Pool instance
# is freed from memory, at each iteration of the main for loop).
chunked_list = chunks(vol_filelist, ncpu)
i            = 0
n_files      = len(vol_filelist)
#loop through chunks
for flist_slice in chunked_list:
    args_list = [(onefile, radar_name) for onefile in flist_slice]
    with Pool(ncpu) as pool:
        pool.starmap(worker, args_list)
        #update user
        i += ncpu
        print('processed: ' + str(round(i/n_files*100,2)))
        
print('finished')

/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/cpol_processing-1.0-py3.6.egg/cpol_processing/processing/radar_codes.py:248: RuntimeWarning: invalid value encountered in log10
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/cpol_processing-1.0-py3.6.egg/cpol_processing/processing/radar_codes.py:248: RuntimeWarning: invalid value encountered in log10
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/cpol_processing-1.0-py3.6.egg/cpol_processing/processing/radar_codes.py:248: RuntimeWarning: invalid value encountered in log10
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/cpol_processing-1.0-py3.6.egg/cpol_processing/processing/radar_codes.py:248: RuntimeWarning: invalid value encountered in log10
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/cpol_processing-1.0-py3.6.egg/cpol_processing/processing/radar_codes.py:248: RuntimeWarning: invalid value encountered in log10
/g/data/kl

completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_043602_processed.nc
completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_044202_processed.nc
completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_044802_processed.nc
completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_045402_processed.nc
completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_050001_processed.nc
completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_050602_processed.nc
completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_051202_processed.nc
completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_051802_processed.nc
completed volume /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_052402_processed.nc
completed volume /g

/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/env

completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_044802_meshgrids.nc
completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_043602_meshgrids.nc
completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_044202_meshgrids.nc


/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log
/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log


...

completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_064802_meshgrids.nc
completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_061202_meshgrids.nc
completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_070001_meshgrids.nc
completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_063003_meshgrids.nc
completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_064202_meshgrids.nc


/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log


completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_063602_meshgrids.nc


/g/data/kl02/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/pyhail-1.0-py3.6.egg/pyhail/mesh.py:131: RuntimeWarning: divide by zero encountered in log


completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_072402_meshgrids.nc
completed grid /g/data1a/rq0/odim_archive/hsda-testing/20141127_processed/CP2_20141127_071802_meshgrids.nc
